# Genomics VEP Pipeline Deployment

This notebook deploys the complete end-to-end genomics VEP processing pipeline.

## Architecture Overview:
1. **VCF Upload** → S3 Input Bucket triggers Lambda
2. **VEP Workflow** → HealthOmics processes VCFs with VEP annotation
3. **EventBridge Monitoring** → Tracks workflow completion
4. **Variant Store Import** → Automatically imports VEP results
5. **Lake Formation Setup** → Creates database permissions for agent queries

## Prerequisites:
- CloudFormation stack deployed using `infrastructure.yaml`
- AWS CLI configured with appropriate permissions
- Files in `./lambda/` directory

## Deployment Steps:
1. Configure parameters
2. Create HealthOmics VEP workflow
3. Create variant and annotation stores
4. Deploy Lambda functions
5. Configure S3 triggers
6. Setup Lake Formation permissions
7. Test the pipeline

In [71]:
import boto3
import json
import time
import zipfile
import base64
import os
import uuid
from datetime import datetime
from pathlib import Path

## Configuration

**IMPORTANT**: Update these parameters to match your CloudFormation stack and requirements.

In [72]:
# AWS Configuration
AWS_PROFILE = '<YOUR_AWS_PROFILE>'  # UPDATE THIS
AWS_REGION = '<YOUR_REGION>'  # UPDATE THIS
CLOUDFORMATION_STACK_NAME = 'genomics-vep-stack'  # UPDATE THIS

# Set AWS profile
os.environ['AWS_PROFILE'] = AWS_PROFILE
session = boto3.Session(profile_name=AWS_PROFILE, region_name=AWS_REGION)

# Initialize clients
cf_client = session.client('cloudformation')
omics_client = session.client('omics')
lambda_client = session.client('lambda')
s3_client = session.client('s3')
iam_client = session.client('iam')
lakeformation = session.client('lakeformation')
glue = session.client('glue')
events_client = session.client('events')
sts_client = session.client('sts')

# Get account info
account_id = sts_client.get_caller_identity()['Account']
print(f"Account ID: {account_id}")
print(f"Region: {AWS_REGION}")
print(f"Profile: {AWS_PROFILE}")

Account ID: <YOUR_ACCOUNT_ID>
Region: <YOUR_REGION>
Profile: default


## Get CloudFormation Stack Outputs

In [73]:
# Get CloudFormation stack outputs
def get_stack_outputs(stack_name):
    try:
        response = cf_client.describe_stacks(StackName=stack_name)
        outputs = {}
        for output in response['Stacks'][0].get('Outputs', []):
            outputs[output['OutputKey']] = output['OutputValue']
        return outputs
    except Exception as e:
        print(f"Error getting stack outputs: {e}")
        return {}

stack_outputs = get_stack_outputs(CLOUDFORMATION_STACK_NAME)
print("CloudFormation Stack Outputs:")
for key, value in stack_outputs.items():
    print(f"  {key}: {value}")

# Extract key values
VCF_INPUT_BUCKET = stack_outputs.get('VcfInputBucketName')
VEP_OUTPUT_BUCKET = stack_outputs.get('VepOutputBucketName')
DYNAMODB_TABLE = stack_outputs.get('DynamoDBTableName')
WORKFLOW_ROLE_ARN = stack_outputs.get('HealthOmicsWorkflowRoleArn')
AGENT_ROLE_ARN = stack_outputs.get('AgentRoleArn')
DATABASE_NAME = stack_outputs.get('DatabaseName')
VCF_PROCESSOR_FUNCTION = stack_outputs.get('VcfProcessorFunctionName')
WORKFLOW_MONITOR_FUNCTION = stack_outputs.get('WorkflowMonitorFunctionName')

CloudFormation Stack Outputs:
  HealthOmicsWorkflowRoleArn: <YOUR_HEALTHOMICS_WORKFLOW_ROLE_ARN>
  WorkflowMonitorFunctionName: genomics-vep-pipeline-workflow-monitor
  KMSKeyArn: <YOUR_KMS_KEY_ARN>
  KMSKeyId: <YOUR_KMS_KEY_ID>
  ReferenceStoreId: <YOUR_REFERENCE_STORE_ID>
  VepOutputBucketName: <YOUR_VEP_OUTPUT_BUCKET>
  DatabaseName: genomicsagentdb2
  VcfProcessorFunctionName: genomics-vep-pipeline-vcf-processor
  ReferenceStoreName: Existing Reference Store
  AgentRoleArn: <YOUR_AGENT_ROLE_ARN>
  LambdaExecutionRoleArn: <YOUR_LAMBDA_EXECUTION_ROLE_ARN>
  DynamoDBTableName: genomics-vep-pipeline-tracking
  VcfInputBucketName: <YOUR_VCF_INPUT_BUCKET>


# Part 2: HealthOmics Workflow Creation

In [7]:
# Configuration for HealthOmics resources
WORKFLOW_NAME = 'vep-workflow2'
VARIANT_STORE_NAME = 'genomicsvariantstore'
ANNOTATION_STORE_NAME = 'genomicsannotationstore'
REFERENCE_STORE_ID = '<YOUR_REFERENCE_STORE_ID>'  # UPDATE your reference store ID
REFERENCE_GENOME_ID = '8931115867' # UPDATE your reference genome ID
BATCH_SIZE = 20

print(f"Workflow Name: {WORKFLOW_NAME}")
print(f"Variant Store: {VARIANT_STORE_NAME}")
print(f"Annotation Store: {ANNOTATION_STORE_NAME}")
print(f"Reference Store ID: {REFERENCE_STORE_ID}")

Workflow Name: vep-workflow2
Variant Store: genomicsvariantstore
Annotation Store: genomicsannotationstore
Reference Store ID: <YOUR_REFERENCE_STORE_ID>


## Create HealthOmics VEP Workflow

In [114]:
def create_vep_workflow():
    """Create VEP workflow from the workflow.zip file"""
    
    # Read and encode the workflow.zip file
    workflow_zip_path = './lambda/workflow.zip'
    
    if not os.path.exists(workflow_zip_path):
        print(f"Error: {workflow_zip_path} not found")
        return None
    
    with open(workflow_zip_path, 'rb') as f:
        workflow_zip_content = f.read()
    
    workflow_zip_base64 = base64.b64encode(workflow_zip_content).decode('utf-8')
    
    # Parameter template for the VEP workflow
    parameter_template = {
        "id": {
            "description": "Sample identifier",
            "optional": False
        },
        "vcf": {
            "description": "Input VCF file path (S3 URI)",
            "optional": False
        },
        "vep_species": {
            "description": "Species name (e.g., homo_sapiens)",
            "optional": False
        },
        "vep_cache": {
            "description": "S3 path to the VEP cache directory",
            "optional": False
        },
        "vep_cache_version": {
            "description": "VEP cache version (e.g., 113)",
            "optional": False
        },
        "ecr_registry": {
            "description": "ECR registry path for container images",
            "optional": False
        },
        "vep_genome": {
            "description": "Reference genome build (e.g., GRCh38)",
            "optional": False
        }
    }
    
    try:
        # Check if workflow already exists
        workflows = omics_client.list_workflows()
        existing_workflow = None
        
        for workflow in workflows['items']:
            if workflow['name'] == WORKFLOW_NAME:
                existing_workflow = workflow
                break
        
        if existing_workflow:
            print(f"Workflow '{WORKFLOW_NAME}' already exists with ID: {existing_workflow['id']}")
            return existing_workflow['id']
        
        # Create new workflow
        print(f"Creating VEP workflow: {WORKFLOW_NAME}")
        
        response = omics_client.create_workflow(
            name=WORKFLOW_NAME,
            description='VEP (Variant Effect Predictor) workflow for genomic variant annotation',
            engine='NEXTFLOW',
            definitionZip=workflow_zip_content,
            parameterTemplate=parameter_template,
            storageCapacity=1200
        )
        
        workflow_id = response['id']
        print(f"Workflow created successfully with ID: {workflow_id}")
        
        # Wait for workflow to be ready
        print("Waiting for workflow to be ready...")
        while True:
            workflow_details = omics_client.get_workflow(id=workflow_id)
            status = workflow_details['status']
            print(f"Workflow status: {status}")
            
            if status == 'ACTIVE':
                print("Workflow is ready!")
                break
            elif status in ['FAILED', 'INACTIVE']:
                print(f"Workflow creation failed with status: {status}")
                if 'statusMessage' in workflow_details:
                    print(f"Error message: {workflow_details['statusMessage']}")
                return None
            
            time.sleep(30)
        
        return workflow_id
        
    except Exception as e:
        print(f"Error creating workflow: {e}")
        return None

# Create the workflow
if __name__ == "__main__":
    WORKFLOW_ID = create_vep_workflow()
    print(f"\nVEP Workflow ID: {WORKFLOW_ID}")

Creating VEP workflow: vep-workflow2
Workflow created successfully with ID: 8533904
Waiting for workflow to be ready...
Workflow status: CREATING
Workflow status: ACTIVE
Workflow is ready!

VEP Workflow ID: 8533904


## Create Variant Store & Annotation Store

In [92]:
def create_variant_store():
    """Create HealthOmics variant store - simple creation only"""
    
    try:
        # Check if variant store already exists
        variant_stores = omics_client.list_variant_stores()
        for store in variant_stores['variantStores']:
            if store['name'] == VARIANT_STORE_NAME:
                print(f"✅ Variant store '{VARIANT_STORE_NAME}' already exists with ID: {store['id']}")
                return store['id']
        
        # Create new variant store
        print(f"🚀 Creating variant store: {VARIANT_STORE_NAME}")
        
        response = omics_client.create_variant_store(
            name=VARIANT_STORE_NAME,
            description='Variant store for genomics VEP pipeline',
            reference={
                'referenceArn': f'arn:aws:omics:{AWS_REGION}:{account_id}:referenceStore/{REFERENCE_STORE_ID}/reference/{REFERENCE_GENOME_ID}'
            }
        )
        
        variant_store_id = response['id']
        print(f"✅ Variant store creation initiated with ID: {variant_store_id}")
        print("⏳ Store is being created in the background...")
        return variant_store_id
        
    except Exception as e:
        print(f"❌ Error creating variant store: {e}")
        return None

def create_annotation_store():
    """Create HealthOmics annotation store - simple creation only"""
    
    try:
        # Check if annotation store already exists
        annotation_stores = omics_client.list_annotation_stores()
        for store in annotation_stores['annotationStores']:
            if store['name'] == ANNOTATION_STORE_NAME:
                print(f"✅ Annotation store '{ANNOTATION_STORE_NAME}' already exists with ID: {store['id']}")
                return store['id']
        
        # Create new annotation store
        print(f"🚀 Creating annotation store: {ANNOTATION_STORE_NAME}")
        
        response = omics_client.create_annotation_store(
            name=ANNOTATION_STORE_NAME,
            description='Annotation store for genomics VEP pipeline',
            storeFormat='VCF',
            reference={
                'referenceArn': f'arn:aws:omics:{AWS_REGION}:{account_id}:referenceStore/{REFERENCE_STORE_ID}/reference/{REFERENCE_GENOME_ID}'
            }
        )
        
        annotation_store_id = response['id']
        print(f"✅ Annotation store creation initiated with ID: {annotation_store_id}")
        print("⏳ Store is being created in the background...")
        return annotation_store_id
        
    except Exception as e:
        print(f"❌ Error creating annotation store: {e}")
        return None

# Create the stores
VARIANT_STORE_ID = create_variant_store()
ANNOTATION_STORE_ID = create_annotation_store()

print(f"\n📋 Summary:")
print(f"Variant Store ID: {VARIANT_STORE_ID}")
print(f"Annotation Store ID: {ANNOTATION_STORE_ID}")
print(f"\n💡 Use the status checker below to monitor progress")

🚀 Creating variant store: genomicsvariantstore
✅ Variant store creation initiated with ID: 049dae23d856
⏳ Store is being created in the background...
🚀 Creating annotation store: genomicsannotationstore
✅ Annotation store creation initiated with ID: 78ff105a1da5
⏳ Store is being created in the background...

📋 Summary:
Variant Store ID: 049dae23d856
Annotation Store ID: 78ff105a1da5

💡 Use the status checker below to monitor progress


In [94]:
def quick_status_check(store_name_or_id):
    try:
        store = omics_client.get_variant_store(name=store_name_or_id)
        return f"Status: {store['status']} | ID: {store['id']}"
    except:
        return "Store not found or still creating"

# Usage
print(quick_status_check('genomicsvariantstore'))

Status: ACTIVE | ID: 049dae23d856


In [95]:
def quick_status_check(store_name_or_id):
    try:
        store = omics_client.get_annotation_store(name=store_name_or_id)
        return f"Status: {store['status']} | ID: {store['id']}"
    except:
        return "Store not found or still creating"

# Usage
print(quick_status_check('genomicsannotationstore'))

Status: ACTIVE | ID: 78ff105a1da5


# Part 3: Lambda Function Deployment

## Deploy VCF Processor Lambda Function

In [96]:
def deploy_vcf_processor_lambda():
    """Deploy the VCF processor Lambda function with proper update handling"""
    
    lambda_zip_path = './lambda/genomics-vep-pipeline-vcf-processor-comprehensive.zip'
    
    if not os.path.exists(lambda_zip_path):
        print(f"Error: {lambda_zip_path} not found")
        return False
    
    try:
        # Read the Lambda function code
        with open(lambda_zip_path, 'rb') as f:
            lambda_zip_content = f.read()
        
        # Step 1: Update the Lambda function code
        print(f"🔄 Updating Lambda function code: {VCF_PROCESSOR_FUNCTION}")
        
        response = lambda_client.update_function_code(
            FunctionName=VCF_PROCESSOR_FUNCTION,
            ZipFile=lambda_zip_content
        )
        
        print(f"✅ Lambda function code updated successfully")
        
        # Step 2: Wait for the code update to complete before updating configuration
        print("⏳ Waiting for code update to complete...")
        
        max_attempts = 30  # 5 minutes max
        for attempt in range(max_attempts):
            try:
                function_info = lambda_client.get_function(FunctionName=VCF_PROCESSOR_FUNCTION)
                last_update_status = function_info['Configuration']['LastUpdateStatus']
                
                print(f"   Attempt {attempt + 1}: Status = {last_update_status}")
                
                if last_update_status == 'Successful':
                    print("✅ Code update completed successfully")
                    break
                elif last_update_status == 'Failed':
                    print("❌ Code update failed")
                    return False
                elif last_update_status in ['InProgress']:
                    print("   Still updating code...")
                else:
                    print(f"   Unknown status: {last_update_status}")
                
            except Exception as e:
                print(f"   Error checking status: {e}")
            
            time.sleep(10)  # Wait 10 seconds between checks
        else:
            print("⚠️  Timeout waiting for code update to complete")
            print("💡 Trying configuration update anyway...")
        
        # Step 3: Update environment variables
        print("🔧 Updating environment variables...")
        
        env_vars = {
            'WORKFLOW_ID': str(WORKFLOW_ID) if WORKFLOW_ID else '',
            'ROLE_ARN': WORKFLOW_ROLE_ARN,
            'OUTPUT_URI': f's3://{VEP_OUTPUT_BUCKET}',
            'DYNAMODB_TABLE': DYNAMODB_TABLE,
            'BATCH_SIZE': str(BATCH_SIZE),
            'ALLOWED_PREFIXES': ''  # Empty means all prefixes allowed
        }
        
        # Wait a bit more before configuration update
        time.sleep(5)
        
        # Retry configuration update with exponential backoff
        max_config_attempts = 5
        for config_attempt in range(max_config_attempts):
            try:
                lambda_client.update_function_configuration(
                    FunctionName=VCF_PROCESSOR_FUNCTION,
                    Environment={'Variables': env_vars}
                )
                
                print("✅ Environment variables updated successfully")
                break
                
            except lambda_client.exceptions.ResourceConflictException as e:
                wait_time = 2 ** config_attempt  # Exponential backoff: 1, 2, 4, 8, 16 seconds
                print(f"   Configuration update conflict (attempt {config_attempt + 1}). Waiting {wait_time}s...")
                time.sleep(wait_time)
                
                if config_attempt == max_config_attempts - 1:
                    print("❌ Failed to update configuration after multiple attempts")
                    print("💡 You can update environment variables manually in the AWS Console")
                    return False
            except Exception as e:
                print(f"❌ Error updating configuration: {e}")
                return False
        
        print("Environment variables updated:")
        for key, value in env_vars.items():
            print(f"  {key}: {value}")
        
        return True
        
    except Exception as e:
        print(f"❌ Error deploying VCF processor Lambda: {e}")
        return False

# Deploy VCF processor Lambda
vcf_processor_deployed = deploy_vcf_processor_lambda()
print(f"\n📊 VCF Processor Lambda deployed: {vcf_processor_deployed}")

🔄 Updating Lambda function code: genomics-vep-pipeline-vcf-processor
✅ Lambda function code updated successfully
⏳ Waiting for code update to complete...
   Attempt 1: Status = InProgress
   Still updating code...
   Attempt 2: Status = Successful
✅ Code update completed successfully
🔧 Updating environment variables...
✅ Environment variables updated successfully
Environment variables updated:
  WORKFLOW_ID: <YOUR_WORKFLOW_ID>
  ROLE_ARN: <YOUR_HEALTHOMICS_WORKFLOW_ROLE_ARN>
  OUTPUT_URI: s3://<YOUR_VEP_OUTPUT_BUCKET>/
  DYNAMODB_TABLE: genomics-vep-pipeline-tracking
  BATCH_SIZE: 20
  ALLOWED_PREFIXES: 

📊 VCF Processor Lambda deployed: True


## Deploy Workflow Monitor Lambda Function

In [97]:
def deploy_workflow_monitor_lambda():
    """Deploy the workflow monitor Lambda function with proper update handling"""
    
    lambda_zip_path = './lambda/genomics-vep-pipeline-workflow-monitor-fixed.zip'
    
    if not os.path.exists(lambda_zip_path):
        print(f"Error: {lambda_zip_path} not found")
        return False
    
    try:
        # Read the Lambda function code
        with open(lambda_zip_path, 'rb') as f:
            lambda_zip_content = f.read()
        
        # Step 1: Update the Lambda function code
        print(f"🔄 Updating Lambda function code: {WORKFLOW_MONITOR_FUNCTION}")
        
        response = lambda_client.update_function_code(
            FunctionName=WORKFLOW_MONITOR_FUNCTION,
            ZipFile=lambda_zip_content
        )
        
        print(f"✅ Lambda function code updated successfully")
        
        # Step 2: Wait for the code update to complete before updating configuration
        print("⏳ Waiting for code update to complete...")
        
        max_attempts = 30  # 5 minutes max
        for attempt in range(max_attempts):
            try:
                function_info = lambda_client.get_function(FunctionName=WORKFLOW_MONITOR_FUNCTION)
                last_update_status = function_info['Configuration']['LastUpdateStatus']
                
                print(f"   Attempt {attempt + 1}: Status = {last_update_status}")
                
                if last_update_status == 'Successful':
                    print("✅ Code update completed successfully")
                    break
                elif last_update_status == 'Failed':
                    print("❌ Code update failed")
                    return False
                elif last_update_status in ['InProgress']:
                    print("   Still updating code...")
                else:
                    print(f"   Unknown status: {last_update_status}")
                
            except Exception as e:
                print(f"   Error checking status: {e}")
            
            time.sleep(10)  # Wait 10 seconds between checks
        else:
            print("⚠️  Timeout waiting for code update to complete")
            print("💡 Trying configuration update anyway...")
        
        # Step 3: Update environment variables
        print("🔧 Updating environment variables...")
        
        env_vars = {
            'ROLE_ARN': WORKFLOW_ROLE_ARN,
            'VARIANT_STORE_NAME': VARIANT_STORE_NAME,
            'ANNOTATION_STORE_NAME': ANNOTATION_STORE_NAME,
            'DYNAMODB_TABLE': DYNAMODB_TABLE,
            'DATABASE_NAME': DATABASE_NAME
        }
        
        # Wait a bit more before configuration update
        time.sleep(5)
        
        # Retry configuration update with exponential backoff
        max_config_attempts = 5
        for config_attempt in range(max_config_attempts):
            try:
                lambda_client.update_function_configuration(
                    FunctionName=WORKFLOW_MONITOR_FUNCTION,
                    Environment={'Variables': env_vars}
                )
                
                print("✅ Environment variables updated successfully")
                break
                
            except lambda_client.exceptions.ResourceConflictException as e:
                wait_time = 2 ** config_attempt  # Exponential backoff: 1, 2, 4, 8, 16 seconds
                print(f"   Configuration update conflict (attempt {config_attempt + 1}). Waiting {wait_time}s...")
                time.sleep(wait_time)
                
                if config_attempt == max_config_attempts - 1:
                    print("❌ Failed to update configuration after multiple attempts")
                    print("💡 You can update environment variables manually in the AWS Console")
                    return False
            except Exception as e:
                print(f"❌ Error updating configuration: {e}")
                return False
        
        print("Environment variables updated:")
        for key, value in env_vars.items():
            print(f"  {key}: {value}")
        
        return True
        
    except Exception as e:
        print(f"❌ Error deploying workflow monitor Lambda: {e}")
        return False

# Deploy workflow monitor Lambda
workflow_monitor_deployed = deploy_workflow_monitor_lambda()
print(f"\n📊 Workflow Monitor Lambda deployed: {workflow_monitor_deployed}")

🔄 Updating Lambda function code: genomics-vep-pipeline-workflow-monitor
✅ Lambda function code updated successfully
⏳ Waiting for code update to complete...
   Attempt 1: Status = InProgress
   Still updating code...
   Attempt 2: Status = Successful
✅ Code update completed successfully
🔧 Updating environment variables...
✅ Environment variables updated successfully
Environment variables updated:
  ROLE_ARN: <YOUR_HEALTHOMICS_WORKFLOW_ROLE_ARN>
  VARIANT_STORE_NAME: genomicsvariantstore
  ANNOTATION_STORE_NAME: genomicsannotationstore
  DYNAMODB_TABLE: genomics-vep-pipeline-tracking
  DATABASE_NAME: genomicsagentdb2

📊 Workflow Monitor Lambda deployed: True


## Configure S3 Event Notifications

In [98]:
def configure_s3_notifications_fixed():
    """Configure S3 event notifications with correct parameter names"""
    
    try:
        # 🔹 INPUT BUCKET CONFIGURATION
        print(f"Configuring S3 notifications for input bucket: {VCF_INPUT_BUCKET}")
        
        vcf_input_notification = {
            'LambdaFunctionConfigurations': [  # ✅ Fixed: was 'LambdaConfigurations'
                {
                    'Id': 'VcfProcessorTrigger',
                    'LambdaFunctionArn': f'arn:aws:lambda:{AWS_REGION}:{account_id}:function:{VCF_PROCESSOR_FUNCTION}',
                    'Events': ['s3:ObjectCreated:*'],
                    'Filter': {
                        'Key': {
                            'FilterRules': [
                                {'Name': 'suffix', 'Value': '.vcf'}
                            ]
                        }
                    }
                },
                {
                    'Id': 'VcfGzProcessorTrigger',
                    'LambdaFunctionArn': f'arn:aws:lambda:{AWS_REGION}:{account_id}:function:{VCF_PROCESSOR_FUNCTION}',
                    'Events': ['s3:ObjectCreated:*'],
                    'Filter': {
                        'Key': {
                            'FilterRules': [
                                {'Name': 'suffix', 'Value': '.vcf.gz'}
                            ]
                        }
                    }
                }
            ]
        }
        
        s3_client.put_bucket_notification_configuration(
            Bucket=VCF_INPUT_BUCKET,
            NotificationConfiguration=vcf_input_notification
        )
        
        print("✅ VCF input bucket notifications configured successfully")
        
        # 🔹 OUTPUT BUCKET CONFIGURATION
        print(f"Configuring S3 notifications for output bucket: {VEP_OUTPUT_BUCKET}")
        
        vep_output_notification = {
            'LambdaFunctionConfigurations': [  # ✅ Fixed: was 'LambdaConfigurations'
                {
                    'Id': 'VepAnnotatedVcfTrigger',
                    'LambdaFunctionArn': f'arn:aws:lambda:{AWS_REGION}:{account_id}:function:{WORKFLOW_MONITOR_FUNCTION}',
                    'Events': ['s3:ObjectCreated:*'],
                    'Filter': {
                        'Key': {
                            'FilterRules': [
                                {'Name': 'suffix', 'Value': '.ann.vcf.gz'}  # VEP annotated files
                            ]
                        }
                    }
                },
                {
                    'Id': 'VepAnnotatedVcfUncompressedTrigger',
                    'LambdaFunctionArn': f'arn:aws:lambda:{AWS_REGION}:{account_id}:function:{WORKFLOW_MONITOR_FUNCTION}',
                    'Events': ['s3:ObjectCreated:*'],
                    'Filter': {
                        'Key': {
                            'FilterRules': [
                                {'Name': 'suffix', 'Value': '.ann.vcf'}  # VEP annotated files (uncompressed)
                            ]
                        }
                    }
                }
            ]
        }
        
        s3_client.put_bucket_notification_configuration(
            Bucket=VEP_OUTPUT_BUCKET,
            NotificationConfiguration=vep_output_notification
        )
        
        print("✅ VEP output bucket notifications configured successfully")
        
        return True
        
    except Exception as e:
        print(f"❌ Error configuring S3 notifications: {e}")
        return False

## Configure EventBridge monitoring

In [99]:
def configure_eventbridge_workflow_monitoring():
    """Configure EventBridge to monitor HealthOmics workflow completion"""
    
    try:
        # Check if EventBridge rule already exists
        existing_rules = events_client.list_rules(NamePrefix=f'{CLOUDFORMATION_STACK_NAME}-workflow-status')
        
        if existing_rules['Rules']:
            print("✅ EventBridge workflow monitoring rule already exists")
            return True
        
        # Create EventBridge rule for workflow completion
        rule_name = f'{CLOUDFORMATION_STACK_NAME}-workflow-completion-monitor'
        
        print(f"🔧 Creating EventBridge rule: {rule_name}")
        
        # Create the rule
        events_client.put_rule(
            Name=rule_name,
            EventPattern=json.dumps({
                "source": ["aws.omics"],
                "detail-type": ["HealthOmics Run Status Change"],
                "detail": {
                    "status": ["COMPLETED", "FAILED", "CANCELLED"],
                    "workflowId": [str(WORKFLOW_ID)] if WORKFLOW_ID else []
                }
            }),
            State='ENABLED',
            Description='Monitor VEP workflow completion for automatic variant import'
        )
        
        # Add Lambda target
        events_client.put_targets(
            Rule=rule_name,
            Targets=[
                {
                    'Id': '1',
                    'Arn': f'arn:aws:lambda:{AWS_REGION}:{account_id}:function:{WORKFLOW_MONITOR_FUNCTION}',
                    'InputTransformer': {
                        'InputPathsMap': {
                            'runId': '$.detail.runId',
                            'status': '$.detail.status',
                            'workflowId': '$.detail.workflowId'
                        },
                        'InputTemplate': '{"source": "eventbridge", "runId": "<runId>", "status": "<status>", "workflowId": "<workflowId>"}'
                    }
                }
            ]
        )
        
        print("✅ EventBridge workflow monitoring configured successfully")
        return True
        
    except Exception as e:
        print(f"❌ Error configuring EventBridge: {e}")
        return False


## Checking the configurations

In [100]:
def configure_complete_pipeline_triggers():
    """Configure both S3 and EventBridge triggers for complete pipeline"""
    
    print("🚀 Configuring Complete Pipeline Triggers")
    print("=" * 50)
    
    # 1. Configure S3 notifications for VCF input (triggers VEP workflow)
    print("1️⃣ Configuring S3 notifications for VCF input...")
    s3_success = configure_s3_notifications_fixed()
    
    # 2. Configure EventBridge for workflow completion (triggers variant import)
    print("\n2️⃣ Configuring EventBridge for workflow monitoring...")
    eventbridge_success = configure_eventbridge_workflow_monitoring()
    
    # 3. Optional: Configure S3 notifications for VEP output (backup trigger)
    print("\n3️⃣ S3 notifications for VEP output configured as backup")
    
    print(f"\n📊 Configuration Summary:")
    print(f"   S3 Input Triggers: {'✅ Success' if s3_success else '❌ Failed'}")
    print(f"   EventBridge Monitoring: {'✅ Success' if eventbridge_success else '❌ Failed'}")
    
    print(f"\n🔄 Pipeline Flow:")
    print(f"   1. Upload VCF → S3 Input Bucket → VCF Processor Lambda → VEP Workflow")
    print(f"   2. VEP Workflow Complete → EventBridge → Workflow Monitor Lambda → Variant Import")
    print(f"   3. Variant Import Complete → Data ready for Agent queries")
    
    return s3_success and eventbridge_success

# Configure the complete pipeline
pipeline_configured = configure_complete_pipeline_triggers()

🚀 Configuring Complete Pipeline Triggers
1️⃣ Configuring S3 notifications for VCF input...
Configuring S3 notifications for input bucket: <YOUR_VCF_INPUT_BUCKET>
✅ VCF input bucket notifications configured successfully
Configuring S3 notifications for output bucket: <YOUR_VEP_OUTPUT_BUCKET>
✅ VEP output bucket notifications configured successfully

2️⃣ Configuring EventBridge for workflow monitoring...
🔧 Creating EventBridge rule: genomics-vep-stack-workflow-completion-monitor
✅ EventBridge workflow monitoring configured successfully

3️⃣ S3 notifications for VEP output configured as backup

📊 Configuration Summary:
   S3 Input Triggers: ✅ Success
   EventBridge Monitoring: ✅ Success

🔄 Pipeline Flow:
   1. Upload VCF → S3 Input Bucket → VCF Processor Lambda → VEP Workflow
   2. VEP Workflow Complete → EventBridge → Workflow Monitor Lambda → Variant Import
   3. Variant Import Complete → Data ready for Agent queries


# Part 3b: Running analytics on healthomics variant store

### Python package imports

In [8]:
from datetime import datetime
import json
import os
import time
import urllib

import boto3
import botocore.exceptions

### Create a service IAM role

For the purposes of this tutorial, we will use the following policy and trust policy to demo the capabilities of AWS HealthOmics, you are free to customize permissions as required for your use case after going though this tutorial.

NOTE: In this case we've defined rather permissive permissions (i.e. "*" Resources). In particular, we are allowing read/write access to all S3 buckets available to the account for this tutorial. In a real world setting you will want to scope this down to only the minimally needed actions on necessary resources.

In [9]:
# set a timestamp
dt_fmt = '%Y%m%dT%H%M%S'
ts = datetime.now().strftime(dt_fmt)

In [193]:
demo_policy = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "omics:*"
      ],
      "Resource": "*"
    },
    {
      "Effect": "Allow",
      "Action": [
        "ram:AcceptResourceShareInvitation",
        "ram:GetResourceShareInvitations"
      ],
      "Resource": "*"
    },
    {
      "Effect": "Allow",
      "Action": [
        "s3:GetBucketLocation",
        "s3:PutObject",
        "s3:GetObject",
        "s3:ListBucket",
        "s3:AbortMultipartUpload",
        "s3:ListMultipartUploadParts",
        "s3:GetObjectAcl",
        "s3:PutObjectAcl"
      ],
      "Resource": "*"
    }
  ]
}

demo_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "omics.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

In order to proceed we need to create a couple of resources the first is the role that you will be passing into AWS HealthOmics. If the role doesn't exist, we will need to create it and attach the policy and trust policy defined above.

In [194]:
# We will use this as the base name for our role and policy
omics_iam_name = f'OmicsTutorialRole-{ts}'

# Create the iam client
iam = boto3.resource('iam')

# Check if the role already exist if not create it
try:
    role = iam.Role(omics_iam_name)
    role.load()
    
except botocore.exceptions.ClientError as ex:
    if ex.response["Error"]["Code"] == "NoSuchEntity":
        #Create the role with the corresponding trust policy
        role = iam.create_role(
            RoleName=omics_iam_name, 
            AssumeRolePolicyDocument=json.dumps(demo_trust_policy))
        
        #Create policy
        policy = iam.create_policy(
            PolicyName='{}-policy'.format(omics_iam_name), 
            Description="Policy for AWS HealthOmics demo",
            PolicyDocument=json.dumps(demo_policy))
        
        #Attach the policy to the role
        policy.attach_role(RoleName=omics_iam_name)
    else:
        print('Something went wrong, please retry and check your account settings and permissions')
        print(ex)

Now that we know the role exists, lets create a helper function to help us retrieve the role arn which we will need to pass into the service API calls. The role arn will grant AWS HealthOmics the permissions it needs to access the resources it needs in your AWS account.

In [10]:
omics_iam_name = 'OmicsTutorialRole-20250818T091459'

In [11]:
def get_role_arn(role_name):
    try:
        iam = boto3.resource('iam')
        role = iam.Role(role_name)
        role.load()  # calls GetRole to load attributes
    except ClientError:
        print("Couldn't get role named %s."%role_name)
        raise
    else:
        return role.arn

### Creating the AWS HealthOmics client

In [1]:
import boto3
omics = boto3.client('omics')

region = omics.meta.region_name
regional_bucket = f'<YOUR_SOURCE_BUCKET>'

### Check the created variant store and vcf files imported into it

In [74]:
var_store = omics.get_variant_store(name='genomicsvariantstore')

In [199]:
omics.list_variant_import_jobs(filter={"storeName": var_store['name']})

{'ResponseMetadata': {'RequestId': '<YOUR_JOB_ID>',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 18 Aug 2025 09:24:23 GMT',
   'content-type': 'application/json',
   'content-length': '1433',
   'connection': 'keep-alive',
   'x-amzn-requestid': '<YOUR_JOB_ID>',
   'x-amz-apigw-id': 'PfqjSHj_IAMErAg=',
   'x-amzn-trace-id': 'Root=1-68a2f147-5a0a8f8743ad961b6555272a'},
  'RetryAttempts': 0},
 'variantImportJobs': [{'id': '<YOUR_JOB_ID>',
   'destinationName': 'genomicsvariantstore',
   'roleArn': '<YOUR_HEALTHOMICS_WORKFLOW_ROLE_ARN>',
   'status': 'COMPLETED',
   'creationTime': datetime.datetime(2025, 8, 17, 5, 54, 52, 492973, tzinfo=tzlocal()),
   'updateTime': datetime.datetime(2025, 8, 17, 6, 36, 47, 991812, tzinfo=tzlocal()),
   'runLeftNormalization': True,
   'annotationFields': {'VEP': 'CSQ'}},
  {'id': '<YOUR_JOB_ID>',
   'destinationName': 'genomicsvariantstore',
   'roleArn': '<YOUR_HEALTHOMICS_WORKFLOW_ROLE_ARN>',
   'status': 'FAILED',
   'creationTime': dateti

## Step 4 - Query in Athena

In order to query your data in Amazon Athena, you need to create resource links to your database using the AWS Lake Formation Console. You will also need to ensure that the IAM user running this notebook is a Data Lake Administrator. Note without both of these in place, the following queries will fail. To satisfy these prerequisites, refer to the instructions provided in the AWS Lake Formation documentation and AWS HealthOmics documentation.

The following code will create resource links to the default database in the AwsDataCatalog in AWS Glue. It makes a few assumptions to do so - like IAM identity you are using to run this notebook is a Data Lake Administrator and has the permissions to create AWS Glue tables.

If you want to be fully sure you are making the correct resource links and providing access to them to the correct identities it is best to create them directly. Refer to the instructions in the AWS HealthOmics documentation on how to do this.

We'll need to work with AWS RAM, AWS Glue, and AWS Lake Formation to setup resource links and grant database permissions.

In [200]:
ram = boto3.client('ram')
glue = boto3.client('glue')

caller_identity = boto3.client('sts').get_caller_identity()
AWS_ACCOUNT_ID = caller_identity['Account']
AWS_IDENITY_ARN = caller_identity['Arn']

First we'll list available shared resources from OTHER-ACCOUNTS in AWS RAM and look for the resource that matches the id of the Variant store we created above.

In [201]:
response = ram.list_resources(resourceOwner='OTHER-ACCOUNTS', resourceType='glue:Database')

if not response.get('resources'):
    print('no shared resources found. verify that you have successfully created an HealthOmics Analytics store')
else:
    variantstore_resources = [resource for resource in response['resources'] if var_store['id'] in resource['arn']]
    if not variantstore_resources:
        print(f"no shared resources matching variant store id {var_store['id']} found")
    else:
        variantstore_resource = variantstore_resources[0]

variantstore_resource

{'arn': '<YOUR_GLUE_DATABASE_ARN>',
 'type': 'glue:Database',
 'resourceShareArn': '<YOUR_RESOURCE_SHARE_ARN>',
 'status': 'AVAILABLE',
 'creationTime': datetime.datetime(2025, 8, 14, 21, 29, 59, 923000, tzinfo=tzlocal()),
 'lastUpdatedTime': datetime.datetime(2025, 8, 14, 21, 30, 3, 336000, tzinfo=tzlocal()),
 'resourceRegionScope': 'REGIONAL'}

Next, we'll get the specific resource share the Variant store is associated with. This is so we can get the owningAccountId attribute for the share. (Note we could also do this by parsing the resourceShareArn for the resource above, but doing it this way is more explicit)

In [202]:
resource_share = ram.get_resource_shares(
    resourceOwner='OTHER-ACCOUNTS', 
    resourceShareArns=[variantstore_resource['resourceShareArn']])['resourceShares'][0]
resource_share

{'resourceShareArn': '<YOUR_RESOURCE_SHARE_ARN>',
 'name': 'LakeFormation-V2-1YNBPN1FFB',
 'owningAccountId': '<YOUR_SHARED_ACCOUNT_ID>',
 'allowExternalPrincipals': True,
 'status': 'ACTIVE',
 'creationTime': datetime.datetime(2025, 7, 18, 6, 51, 27, 393000, tzinfo=tzlocal()),
 'lastUpdatedTime': datetime.datetime(2025, 7, 18, 6, 51, 27, 393000, tzinfo=tzlocal()),
 'featureSet': 'STANDARD'}

Now we'll create a table in AWS Glue for the variant store. This is the same as creating a resource link in AWS Lake Formation.

In [203]:
# this creates a resource link to the table for the variant store and adds it to the `default` database
glue.create_table(
    DatabaseName='<YOUR_AWS_PROFILE>',
    TableInput = {
        "Name": var_store['name'],
        "TargetTable": {
            "CatalogId": resource_share['owningAccountId'],
            "DatabaseName": f"variant_{AWS_ACCOUNT_ID}_{var_store['id']}",
            "Name": var_store['name'],
        }
    }
)

{'ResponseMetadata': {'RequestId': '<YOUR_JOB_ID>',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 18 Aug 2025 09:28:25 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '<YOUR_JOB_ID>',
   'cache-control': 'no-cache'},
  'RetryAttempts': 0}}

In [14]:
!aws sts get-caller-identity

{
    "UserId": "<YOUR_USER_ID>",
    "Account": "<YOUR_ACCOUNT_ID>",
    "Arn": "<YOUR_ASSUMED_ROLE_ARN>"
}


For this section of the tutorial, the identity that runs this notebook either:

1. needs to be a Data lake administrator in AWS Lake Formation, or
2. must be granted access to the AWS RAM shared resources by an existing administrator.
The latter pattern is recommended. Both DESCRIBE and SELECT on the target table for the variant store are required and can be done via the "Grant on target" action on a resource link in the AWS Lake Formation Console. You can also do this with and admin identity via the SDK with code like:

In [ ]:
sts = boto3.client('sts')
identity = sts.get_caller_identity()

# Extract the role ARN from the assumed role ARN
assumed_role_arn = identity['Arn']
# Convert from: <YOUR_ASSUMED_ROLE_ARN>
# To: arn:aws:iam::<YOUR_ACCOUNT_ID>:role/SageMakerNotebookInstanceRole
role_arn = assumed_role_arn.replace(':sts:', ':iam:').replace(':assumed-role/', ':role/').rsplit('/', 1)[0]

lfn = boto3.client('lakeformation')

lfn.grant_permissions(
    Principal={
        "DataLakePrincipalIdentifier": role_arn
    },
    Resource={
        "Table": {
            "CatalogId": resource_share['owningAccountId'],
            "DatabaseName": f"variant_{AWS_ACCOUNT_ID}_{var_store['id']}",
            "Name": var_store['name']
        }
    },
    Permissions=[ 'DESCRIBE' ],
    PermissionsWithGrantOption=[ 'DESCRIBE' ]
)

lfn.grant_permissions(
    Principal={
        "DataLakePrincipalIdentifier": role_arn
    },
    Resource={
        "TableWithColumns": {
            "CatalogId": resource_share['owningAccountId'],
            "DatabaseName": f"variant_{AWS_ACCOUNT_ID}_{var_store['id']}",
            "Name": var_store['name'],
            "ColumnWildcard": {}
        }
    },
    Permissions=[ 'SELECT' ],
    PermissionsWithGrantOption=[ 'SELECT' ]
)

Now that we have resource links created, we can start querying the data using Amazon Athena. You don't need to wait for all the import jobs to complete to start doing this. Queries can be made while data imports in the background.

To query AWS HealthOmics Analytics stores, you need to use Athena engine version 3. The following code checks if you have an existing Athena workgroup that satisfies this criteria. If not it will create one called omics.

In [3]:
athena = boto3.client('athena')

In [4]:
athena_workgroups = athena.list_work_groups()['WorkGroups']
athena_workgroups

[{'Name': 'genomicsanalysis-<YOUR_REGION>',
  'State': 'ENABLED',
  'Description': 'genomicsanalysis',
  'CreationTime': datetime.datetime(2023, 3, 21, 13, 30, 11, 400000, tzinfo=tzlocal()),
  'EngineVersion': {'SelectedEngineVersion': 'Athena engine version 3',
   'EffectiveEngineVersion': 'Athena engine version 3'}},
 {'Name': 'primary',
  'State': 'ENABLED',
  'Description': '',
  'CreationTime': datetime.datetime(2022, 9, 22, 11, 33, 40, 695000, tzinfo=tzlocal()),
  'EngineVersion': {'SelectedEngineVersion': 'AUTO',
   'EffectiveEngineVersion': 'Athena engine version 3'}}]

In [5]:
athena_workgroups = athena.list_work_groups()['WorkGroups']

athena_workgroup = None
for wg in athena_workgroups:
    print(wg['EngineVersion']['EffectiveEngineVersion'])
    if wg['EngineVersion']['EffectiveEngineVersion'] == 'Athena engine version 3':
        print(f"Workgroup '{wg['Name']}' found using Athena engine version 3")
        athena_workgroup = wg
        break
else:
    print("No workgroups with Athena engine version 3 found. creating one")
    athena_workgroup = athena.create_work_group(
        Name='omics',
        Configuration={
            "EngineVersion": {
                "SelectedEngineVersion": "Athena engine version 3"
            }
        }
    )

athena_workgroup

Athena engine version 3
Workgroup 'genomicsanalysis-<YOUR_REGION>' found using Athena engine version 3


{'Name': 'genomicsanalysis-<YOUR_REGION>',
 'State': 'ENABLED',
 'Description': 'genomicsanalysis',
 'CreationTime': datetime.datetime(2023, 3, 21, 13, 30, 11, 400000, tzinfo=tzlocal()),
 'EngineVersion': {'SelectedEngineVersion': 'Athena engine version 3',
  'EffectiveEngineVersion': 'Athena engine version 3'}}

Let's start writing queries!

For fun, let's calculate the TI/TV ratio for these samples. You can navigate to the Athena console or do this from your Jupyter Notebook. This example uses the workgroup omics and assumes you have made a resource link to your Variant store in your default database.

In [6]:
import awswrangler as wr

In [18]:
simple_query = f"""SELECT * FROM "default"."{var_store['name']}" LIMIT 10"""

In [216]:
!pip install awswrangler

In [19]:
import awswrangler as wr
df_titv = wr.athena.read_sql_query(
    simple_query, 
    database='<YOUR_AWS_PROFILE>', 
    workgroup=athena_workgroup['Name'])

In [20]:
df_titv

,importjobid,contigname,start,end,names,referenceallele,alternatealleles,qual,filters,splitfrommultiallelic,...,genotypelikelihoods,phredlikelihoods,alleledepths,conditionalquality,spl,depth,ps,sampleid,information,annotations
0,-0,chr8,15479431,15479432,None,T,[C],37.10,[PASS],False,...,None,"[72, 0, 50]","[24, 10]",37,None,34,<NA>,NA21144,"[(mb, [11, 13, 5, 5]), (posteriorprobabilities...","{'vep': [{'allele': 'C', 'consequence': ['intr..."
1,-0,chr14,22556693,22556694,None,C,[T],50.00,[PASS],False,...,None,"[85, 0, 50]","[19, 21]",48,None,40,22556690,NA21144,"[(mb, [10, 9, 12, 9]), (posteriorprobabilities...","{'vep': [{'allele': 'T', 'consequence': ['intr..."
2,-0,chr8,15479756,15479757,None,C,[T],49.41,[PASS],False,...,None,"[84, 0, 50]","[18, 15]",48,None,33,<NA>,NA21144,"[(mb, [7, 11, 6, 9]), (posteriorprobabilities,...","{'vep': [{'allele': 'T', 'consequence': ['intr..."
3,-0,chr8,15476555,15476556,None,A,[G],50.00,[PASS],False,...,None,"[85, 0, 38]","[7, 19]",40,None,26,<NA>,NA21144,"[(mb, [3, 4, 8, 11]), (posteriorprobabilities,...","{'vep': [{'allele': 'G', 'consequence': ['intr..."
4,-0,chr14,22554950,22554951,None,A,[G],40.26,[PASS],False,...,None,"[75, 0, 50]","[19, 8]",40,None,27,<NA>,NA21144,"[(mb, [13, 6, 4, 4]), (posteriorprobabilities,...","{'vep': [{'allele': 'G', 'consequence': ['down..."
5,-0,chr8,15478374,15478375,None,A,[G],49.81,[PASS],False,...,None,"[85, 0, 50]","[20, 18]",48,None,38,<NA>,NA21144,"[(mb, [10, 10, 14, 4]), (posteriorprobabilitie...","{'vep': [{'allele': 'G', 'consequence': ['intr..."
6,-0,chr14,22555623,22555624,None,G,[A],50.00,[PASS],False,...,None,"[85, 0, 50]","[13, 13]",48,None,26,<NA>,NA21144,"[(mb, [6, 7, 7, 6]), (posteriorprobabilities, ...","{'vep': [{'allele': 'A', 'consequence': ['down..."
7,-0,chr8,15478931,15478932,None,C,[A],50.00,[PASS],False,...,None,"[85, 0, 49]","[10, 14]",48,None,24,<NA>,NA21144,"[(mb, [7, 3, 10, 4]), (posteriorprobabilities,...","{'vep': [{'allele': 'A', 'consequence': ['intr..."
8,-0,chr14,22556689,22556691,None,CA,[C],49.50,[PASS],False,...,None,"[79, 0, 50]","[21, 18]",48,None,39,22556690,NA21144,"[(mb, [12, 9, 9, 9]), (posteriorprobabilities,...","{'vep': [{'allele': '-', 'consequence': ['intr..."
9,-0,chr14,22554576,22554577,None,G,[C],48.75,[PASS],False,...,None,"[84, 0, 50]","[16, 12]",47,None,28,<NA>,NA21144,"[(mb, [6, 10, 7, 5]), (posteriorprobabilities,...","{'vep': [{'allele': 'C', 'consequence': ['down..."


In [7]:
count_query = f"""SELECT DISTINCT sampleid FROM "default"."{var_store['name']}" """

In [8]:
df_count = wr.athena.read_sql_query(
    count_query, 
    database='<YOUR_AWS_PROFILE>', 
    workgroup=athena_workgroup['Name'])
df_count

,sampleid
0,NA21135
1,NA21144


In [177]:
!aws s3 cp s3://<YOUR_SOURCE_BUCKET>/dragen_vcfs2/NA21135.hard-filtered.vcf.gz s3://<YOUR_VCF_INPUT_BUCKET>/

copy: s3://<YOUR_SOURCE_BUCKET>/dragen_vcfs2/NA21135.hard-filtered.vcf.gz to s3://<YOUR_VCF_INPUT_BUCKET>/NA21135.hard-filtered.vcf.gz


In [186]:
!aws dynamodb scan --table-name genomics-vep-pipeline-tracking

{
    "Items": [
        {
            "OutputPath": {
                "S": "s3://<YOUR_VEP_OUTPUT_BUCKET>/NA21135/"
            },
            "S3Key": {
                "S": "NA21135.hard-filtered.vcf.gz"
            },
            "BatchID": {
                "S": "330f9db8"
            },
            "S3Bucket": {
                "S": "<YOUR_VCF_INPUT_BUCKET>"
            },
            "Status": {
                "S": "RUNNING"
            },
            "OriginalFilename": {
                "S": "NA21135.hard-filtered.vcf.gz"
            },
            "S3Prefix": {
                "S": ""
            },
            "WorkflowRunID": {
                "S": "1104501"
            },
            "ProcessingStage": {
                "S": "VEP_WORKFLOW"
            },
            "SampleID": {
                "S": "NA21135"
            },
            "BatchNumber": {
                "N": "1"
            },
            "InputVCF": {
                "S": "s3://<YOUR_VCF_INPUT_BUCKET>/NA2

In [175]:
# Delete first item
# !aws dynamodb delete-item --table-name genomics-vep-pipeline-tracking --key '{"SampleID":{"S":"BATCH_SUMMARY_c8c7e09b"}}'


In [178]:
!aws s3 cp s3://<YOUR_SOURCE_BUCKET>/dragen_vcfs2/NA21144.hard-filtered.vcf.gz s3://<YOUR_VCF_INPUT_BUCKET>/

copy: s3://<YOUR_SOURCE_BUCKET>/dragen_vcfs2/NA21144.hard-filtered.vcf.gz to s3://<YOUR_VCF_INPUT_BUCKET>/NA21144.hard-filtered.vcf.gz


# Annotation Store

Now, let's set up an Annotation store.

AWS HealthOmics Annotation stores support annotations in VCF, GFF, and TSV formats. In this tutorial, we import ClinVar annotations which can be downloaded from the NCBI as a VCF file. Imports need to come from an S3 location in the same region, so we'll use a copy in a regional bucket for this tutorial.

In [221]:
!wget https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar_20250810.vcf.gz

--2025-08-18 12:15:30--  https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar_20250810.vcf.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.7, 130.14.250.10, 130.14.250.11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169062439 (161M) [application/x-gzip]
Saving to: ‘clinvar_20250810.vcf.gz’

100%[======================================>] 169,062,439  205MB/s   in 0.8s   

2025-08-18 12:15:31 (205 MB/s) - ‘clinvar_20250810.vcf.gz’ saved [169062439/169062439]



In [222]:
!aws s3 cp clinvar_20250810.vcf.gz s3://<YOUR_SOURCE_BUCKET>/clinvar20250810/

upload: ./clinvar_20250810.vcf.gz to s3://<YOUR_SOURCE_BUCKET>/clinvar20250810/clinvar_20250810.vcf.gz


In [236]:
SOURCE_ANNOTATION_URI = f"s3://<YOUR_SOURCE_BUCKET>/clinvar20250810/clinvar_20250810.vcf.gz"

## Creating, importing data into, and querying an Annotation store

The process of creating, importing data into, and querying an Annotation store is similar to the process you did above for the Variant store, so we'll be brief on the descriptions of each step.

In [21]:
ann_store_name = f'genomicsannotationstore'
ref_name = 'dragen-ref-genome3'  ## Change this reference name to match one you have created if needed

In [238]:
response = omics.start_annotation_import_job(
    destinationName=f'genomicsannotationstore',
    roleArn=get_role_arn(omics_iam_name),
    items=[{"source": SOURCE_ANNOTATION_URI}]
)

In [233]:
response

{'ResponseMetadata': {'RequestId': '<YOUR_JOB_ID>',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 18 Aug 2025 12:36:35 GMT',
   'content-type': 'application/json',
   'content-length': '48',
   'connection': 'keep-alive',
   'x-amzn-requestid': '<YOUR_JOB_ID>',
   'x-amz-apigw-id': 'PgGtAE1noAMEQjA=',
   'x-amzn-trace-id': 'Root=1-68a31e52-5a788abb3a6435d007f7a98b'},
  'RetryAttempts': 0},
 'jobId': '<YOUR_JOB_ID>'}

In [244]:
omics.list_annotation_import_jobs(filter={"storeName": ann_store_name})

{'ResponseMetadata': {'RequestId': '<YOUR_JOB_ID>',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 18 Aug 2025 12:55:38 GMT',
   'content-type': 'application/json',
   'content-length': '1172',
   'connection': 'keep-alive',
   'x-amzn-requestid': '<YOUR_JOB_ID>',
   'x-amz-apigw-id': 'PgJfsGsKoAMEcTg=',
   'x-amzn-trace-id': 'Root=1-68a322ca-7c02c8a9479339d7133fa762'},
  'RetryAttempts': 0},
 'annotationImportJobs': [{'id': '<YOUR_JOB_ID>',
   'destinationName': 'genomicsannotationstore',
   'versionName': 'genomicsannotationstore',
   'roleArn': '<YOUR_OMICS_TUTORIAL_ROLE_ARN>',
   'status': 'FAILED',
   'creationTime': datetime.datetime(2025, 8, 18, 12, 36, 35, 533482, tzinfo=tzlocal()),
   'updateTime': datetime.datetime(2025, 8, 18, 12, 36, 36, 285605, tzinfo=tzlocal()),
   'runLeftNormalization': False,
   'annotationFields': {'VEP': 'CSQ'}},
  {'id': '<YOUR_JOB_ID>',
   'destinationName': 'genomicsannotationstore',
   'versionName': 'genomicsannotationstore',
   'roleA

In [9]:
ann_store = omics.get_annotation_store(name='genomicsannotationstore')

Creating a resource link to the Annotation store is the same as with the Variant store. We'll do this all in one cell below.

In [246]:
response = ram.list_resources(resourceOwner='OTHER-ACCOUNTS', resourceType='glue:Database')

if not response.get('resources'):
    print('no shared resources found. verify that you have successfully created an HealthOmics Analytics store')
else:
    annotationstore_resources = [resource for resource in response['resources'] if ann_store['id'] in resource['arn']]
    if not annotationstore_resources:
        print(f"no shared resources matching annotation store id {ann_store['id']} found")
    else:
        annotationstore_resource = annotationstore_resources[0]

        resource_share = ram.get_resource_shares(
            resourceOwner='OTHER-ACCOUNTS', 
            resourceShareArns=[annotationstore_resource['resourceShareArn']])['resourceShares'][0]
        
        # this creates a resource link to the table for the annotation store and adds it to the `default` database
        glue.create_table(
            DatabaseName='<YOUR_AWS_PROFILE>',
            TableInput = {
                "Name": ann_store['name'],
                "TargetTable": {
                    "CatalogId": resource_share['owningAccountId'],
                    "DatabaseName": f"annotation_{AWS_ACCOUNT_ID}_{ann_store['id']}",
                    "Name": ann_store['name'],
                }
            }
        )

In [247]:
response

{'ResponseMetadata': {'RequestId': '<YOUR_JOB_ID>',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 18 Aug 2025 12:58:27 GMT',
   'content-type': 'application/json',
   'content-length': '7448',
   'connection': 'keep-alive',
   'x-amzn-requestid': '<YOUR_JOB_ID>',
   'access-control-allow-origin': '*',
   'access-control-allow-headers': 'Content-Type,X-Amz-Date,Authorization,Date,X-Amz-Target,x-amzn-platform-id,x-amzn-trace-id,X-Api-Key,X-Amz-Security-Token,X-Amz-Content-Sha256,X-Amz-User-Agent,Amz-Sdk-Invocation-Id,Amz-Sdk-Request,Sec-Ch-Ua,Sec-Ch-Ua-Mobile,Sec-Ch-Ua-Platform',
   'x-amz-apigw-id': 'PgJ6GHXLIAMEQjA=',
   'access-control-allow-methods': 'POST',
   'access-control-expose-headers': 'x-amzn-errortype,x-amzn-requestid,x-amzn-errormessage,x-amzn-trace-id,x-amz-apigw-id,date',
   'x-amzn-trace-id': 'Root=1-68a32373-057fba787273b6c2775549ea',
   'access-control-max-age': '86400'},
  'RetryAttempts': 0},
 'resources': [{'arn': '<YOUR_GLUE_DATABASE_ARN>',
   'type': '

In [249]:
annot_query = f"""SELECT * FROM "default"."{ann_store['name']}" LIMIT 10"""

In [250]:
df_annot = wr.athena.read_sql_query(
    annot_query, 
    database='<YOUR_AWS_PROFILE>', 
    workgroup=athena_workgroup['Name'])
df_annot

,importjobid,contigname,start,end,names,referenceallele,alternatealleles,qual,filters,splitfrommultiallelic,...,genotypelikelihoods,phredlikelihoods,alleledepths,conditionalquality,spl,depth,ps,sampleid,information,annotations
0,-0,1,66925,66927,[3385321],AG,[A],NaN,None,False,...,None,None,None,<NA>,None,<NA>,<NA>,<NA>,None,{'vep': None}
1,-0,1,69133,69134,[2205837],A,[G],NaN,None,False,...,None,None,None,<NA>,None,<NA>,<NA>,<NA>,None,{'vep': None}
2,-0,1,69307,69308,[3925305],A,[G],NaN,None,False,...,None,None,None,<NA>,None,<NA>,<NA>,<NA>,None,{'vep': None}
3,-0,1,69313,69314,[3205580],T,[G],NaN,None,False,...,None,None,None,<NA>,None,<NA>,<NA>,<NA>,None,{'vep': None}
4,-0,1,69403,69404,[3925306],T,[C],NaN,None,False,...,None,None,None,<NA>,None,<NA>,<NA>,<NA>,None,{'vep': None}
5,-0,1,69422,69423,[3205581],G,[A],NaN,None,False,...,None,None,None,<NA>,None,<NA>,<NA>,<NA>,None,{'vep': None}
6,-0,1,69580,69581,[2252161],C,[G],NaN,None,False,...,None,None,None,<NA>,None,<NA>,<NA>,<NA>,None,{'vep': None}
7,-0,1,69681,69682,[2396347],G,[A],NaN,None,False,...,None,None,None,<NA>,None,<NA>,<NA>,<NA>,None,{'vep': None}
8,-0,1,69730,69731,[3205582],T,[C],NaN,None,False,...,None,None,None,<NA>,None,<NA>,<NA>,<NA>,None,{'vep': None}
9,-0,1,69768,69769,[2288999],T,[C],NaN,None,False,...,None,None,None,<NA>,None,<NA>,<NA>,<NA>,None,{'vep': None}


In [261]:
clin_query = f"""SELECT 
    v.sampleid,
    v.contigname,
    v.start,
    v.referenceallele,
    v.alternatealleles,
    a.names as clinvar_variation_id,
    -- Extract clinical significance from attributes map
    CASE 
        WHEN a.attributes['CLNSIG'] = 'Pathogenic' THEN 'Pathogenic'
        WHEN a.attributes['CLNSIG'] = 'Likely_pathogenic' THEN 'Likely Pathogenic'
        WHEN a.attributes['CLNSIG'] = 'Uncertain_significance' THEN 'Uncertain Significance'
        WHEN a.attributes['CLNSIG'] = 'Likely_benign' THEN 'Likely Benign'
        WHEN a.attributes['CLNSIG'] = 'Benign' THEN 'Benign'
        ELSE 'Unknown'
    END as clinical_significance,
    -- Extract gene name from GENEINFO (split on colon and take first part)
    CASE 
        WHEN a.attributes['GENEINFO'] IS NOT NULL 
        THEN split_part(a.attributes['GENEINFO'], ':', 1)
        ELSE NULL
    END as gene_name,
    -- Keep full gene info for reference
    a.attributes['GENEINFO'] as full_gene_info
FROM "default"."{var_store['name']}" v
INNER JOIN "default"."{ann_store['name']}" a ON (
    REPLACE(v.contigname, 'chr', '') = a.contigname
    AND v.start = a.start
    AND v.referenceallele = a.referenceallele
    AND v.alternatealleles = a.alternatealleles
)
WHERE a.attributes IS NOT NULL 
    AND a.attributes['CLNSIG'] IS NOT NULL"""

In [262]:
clin_annot = wr.athena.read_sql_query(
    clin_query, 
    database='<YOUR_AWS_PROFILE>', 
    workgroup=athena_workgroup['Name'])
clin_annot

,sampleid,contigname,start,referenceallele,alternatealleles,clinvar_variation_id,clinical_significance,gene_name,full_gene_info
0,NA21135,chr14,49625213,T,[A],[313270],Benign,MGAT2,MGAT2:4247|DNAAF2:55172
1,NA21135,chr14,49654139,C,[T],[1164994],Benign,POLE2,POLE2:5427
2,NA21135,chr14,50138548,A,[AT],[1265951],Benign,SOS2,SOS2:6655
3,NA21135,chr14,50152963,CA,[C],[1254508],Benign,SOS2,SOS2:6655
4,NA21135,chr14,50182210,G,[T],[1226181],Benign,SOS2,SOS2:6655
...,...,...,...,...,...,...,...,...,...
87475,NA21144,chr2,55679416,T,[C],[671250],Likely Benign,PNPT1,PNPT1:87178
87476,NA21144,chr2,55680424,TA,[T],[1293931],Benign,PNPT1,PNPT1:87178
87477,NA21144,chr2,55874690,G,[T],[1226094],Benign,EFEMP1,EFEMP1:2202
87478,NA21144,chr2,55917794,T,[C],[93482],Benign,EFEMP1,EFEMP1:2202


In [258]:
explore_query = f"""SELECT 
    map_keys(a.attributes) as available_keys,
    a.attributes
FROM "default"."{ann_store['name']}" a 
WHERE a.attributes IS NOT NULL
LIMIT 10"""

In [ ]:
explore_annot = wr.athena.read_sql_query(
    explore_query, 
    database='<YOUR_AWS_PROFILE>', 
    workgroup=athena_workgroup['Name'])

In [260]:
explore_annot

,available_keys,attributes
0,"[CLNVC, RS, CLNDISDB, CLNSIG, GENEINFO, CLNSIG...","[(CLNVC, single_nucleotide_variant), (RS, 7767..."
1,"[CLNVC, RS, CLNDISDB, CLNSIG, GENEINFO, CLNSIG...","[(CLNVC, single_nucleotide_variant), (RS, 1948..."
2,"[CLNVC, RS, CLNDISDB, CLNSIG, GENEINFO, CLNSIG...","[(CLNVC, single_nucleotide_variant), (RS, 1410..."
3,"[CLNVC, RS, CLNDISDB, CLNSIG, GENEINFO, CLNSIG...","[(CLNVC, single_nucleotide_variant), (RS, 8793..."
4,"[CLNVC, RS, CLNDISDB, CLNSIG, GENEINFO, CLNSIG...","[(CLNVC, single_nucleotide_variant), (RS, 7919..."
5,"[CLNVC, RS, CLNDISDB, CLNSIG, GENEINFO, CLNSIG...","[(CLNVC, single_nucleotide_variant), (RS, 1948..."
6,"[CLNVC, RS, CLNDISDB, CLNSIG, GENEINFO, CLNSIG...","[(CLNVC, single_nucleotide_variant), (RS, 1948..."
7,"[CLNVC, RS, CLNDISDB, CLNSIG, GENEINFO, CLNSIG...","[(CLNVC, single_nucleotide_variant), (RS, 7734..."
8,"[CLNVC, RS, CLNDISDB, CLNSIG, GENEINFO, CLNSIG...","[(CLNVC, single_nucleotide_variant), (RS, 3099..."
9,"[CLNVC, RS, CLNDISDB, CLNSIG, GENEINFO, CLNSIG...","[(CLNVC, single_nucleotide_variant), (RS, 5384..."


In [7]:
import gc
gc.collect()

# Use the limited version
comprehensive_query_limited = f"""
SELECT 
    v.sampleid,
    v.contigname,
    v.start,
    v.qual,
    a.attributes['CLNSIG'] as clinical_significance,
    CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].symbol END as gene_symbol
FROM "default"."{var_store['name']}" v
LEFT JOIN "default"."{ann_store['name']}" a ON (
    REPLACE(v.contigname, 'chr', '') = a.contigname
    AND v.start = a.start
    AND v.referenceallele = a.referenceallele
    AND v.alternatealleles = a.alternatealleles
)
WHERE v.qual > 50 
LIMIT 100
"""

In [15]:
v_explore_annot = wr.athena.read_sql_query(
    comprehensive_query_limited, 
    database='<YOUR_AWS_PROFILE>', 
    workgroup=athena_workgroup['Name'])

In [16]:
v_explore_annot

,sampleid,contigname,start,qual,clinical_significance,gene_symbol
0,NA21144,chr8,16734439,169.88,<NA>,
1,NA21144,chr8,16735203,125.93,<NA>,
2,NA21144,chr8,16735810,319.19,<NA>,
3,NA21144,chr14,24481958,121.75,<NA>,
4,NA21144,chr14,24482717,124.31,<NA>,
...,...,...,...,...,...,...
95,NA21144,chr14,24797999,134.26,<NA>,
96,NA21144,chr8,17014822,231.83,<NA>,
97,NA21144,chr8,17016760,143.27,<NA>,
98,NA21144,chr14,24798777,122.21,<NA>,


In [13]:
filter_check_query = f"""
SELECT 
    sampleid,
    filters,
    typeof(filters) as filter_type,
    cardinality(filters) as filter_count,
    CASE WHEN cardinality(filters) > 0 THEN filters[1] END as first_filter
FROM "default"."{var_store['name']}"
LIMIT 20
"""

print("Fixed queries created:")
print("1. comprehensive_filtered_fixed - Fixed array handling for filters")
print("2. comprehensive_filtered_permissive - No filter restrictions")
print("3. comprehensive_safe_medium - Safe version with 1000 result limit")
print("4. filter_check_query - Check filter field structure first")
print("")
print("RECOMMENDATION: Run filter_check_query first to understand the filter field structure")

Fixed queries created:
1. comprehensive_filtered_fixed - Fixed array handling for filters
2. comprehensive_filtered_permissive - No filter restrictions
3. comprehensive_safe_medium - Safe version with 1000 result limit
4. filter_check_query - Check filter field structure first

RECOMMENDATION: Run filter_check_query first to understand the filter field structure


In [14]:
vep_filter_query = wr.athena.read_sql_query(
    filter_check_query, 
    database='<YOUR_AWS_PROFILE>', 
    workgroup=athena_workgroup['Name'])
vep_filter_query

,sampleid,filters,filter_type,filter_count,first_filter
0,NA21144,[PASS],array(varchar),1,PASS
1,NA21144,[DRAGENHardQUAL],array(varchar),1,DRAGENHardQUAL
2,NA21144,[PASS],array(varchar),1,PASS
3,NA21144,[PASS],array(varchar),1,PASS
4,NA21144,[PASS],array(varchar),1,PASS
5,NA21144,[PASS],array(varchar),1,PASS
6,NA21144,[PASS],array(varchar),1,PASS
7,NA21144,[PASS],array(varchar),1,PASS
8,NA21144,[PASS],array(varchar),1,PASS
9,NA21144,[PASS],array(varchar),1,PASS


In [16]:
import gc
gc.collect()
## High Priority Variants Only (Smallest Result Set)
comprehensive_high_priority = f"""
SELECT 
    v.sampleid,
    v.contigname,
    v.start,
    v.referenceallele,
    v.alternatealleles,
    v.qual,
    v.depth,
    v.filters,
    
    -- ClinVar Information
    a.attributes['CLNSIG'] as clinical_significance,
    a.attributes['CLNDN'] as disease_name,
    a.attributes['GENEINFO'] as clinvar_gene_info,
    
    -- VEP Annotations
    CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].symbol END as vep_gene_symbol,
    CASE WHEN cardinality(v.annotations.vep) > 0 AND cardinality(v.annotations.vep[1].consequence) > 0 
         THEN v.annotations.vep[1].consequence[1] END as vep_consequence,
    CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].impact END as vep_impact,
    
    -- Quality Metrics
    v.information['af'] as allele_frequency,
    v.qual as quality_score,
    
    -- Clinical Assessment
    CASE 
        WHEN a.attributes['CLNSIG'] IN ('Pathogenic', 'Likely_pathogenic') 
             AND (CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].impact END) IN ('HIGH', 'MODERATE')
        THEN 'High Clinical Concern'
        WHEN a.attributes['CLNSIG'] = 'Uncertain_significance' 
             AND (CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].impact END) = 'HIGH'
        THEN 'Moderate Clinical Concern'
        ELSE 'Review Required'
    END as clinical_assessment

FROM "default"."{var_store['name']}" v
LEFT JOIN "default"."{ann_store['name']}" a ON (
    REPLACE(v.contigname, 'chr', '') = a.contigname
    AND v.start = a.start
    AND v.referenceallele = a.referenceallele
    AND v.alternatealleles = a.alternatealleles
)
WHERE v.qual > 50  
    AND contains(v.filters, 'PASS')  -- Only passed variants
    AND (
        a.attributes['CLNSIG'] IN ('Pathogenic', 'Likely_pathogenic', 'Uncertain_significance')  -- Only clinically significant
        OR (cardinality(v.annotations.vep) > 0 AND v.annotations.vep[1].impact = 'HIGH')  -- Or high impact
    )
ORDER BY 
    CASE 
        WHEN a.attributes['CLNSIG'] = 'Pathogenic' THEN 1
        WHEN a.attributes['CLNSIG'] = 'Likely_pathogenic' THEN 2
        WHEN a.attributes['CLNSIG'] = 'Uncertain_significance' THEN 3
        ELSE 4
    END,
    v.qual DESC
"""

In [17]:
vep_filter_comp = wr.athena.read_sql_query(
    comprehensive_high_priority, 
    database='<YOUR_AWS_PROFILE>', 
    workgroup=athena_workgroup['Name'])
vep_filter_comp

,sampleid,contigname,start,referenceallele,alternatealleles,qual,depth,filters,clinical_significance,disease_name,clinvar_gene_info,vep_gene_symbol,vep_consequence,vep_impact,allele_frequency,quality_score,clinical_assessment
0,NA21144,chr11_KI270902v1_alt,82578,AG,[A],54.60,4,[PASS],<NA>,<NA>,<NA>,MUC6,frameshift_variant,HIGH,[1.0],54.60,Review Required
1,NA21144,chr16_KI270728v1_random,1142711,T,[C],53.10,10,[PASS],<NA>,<NA>,<NA>,,splice_acceptor_variant,HIGH,[1.0],53.10,Review Required
2,NA21135,chr1,228384787,C,[A],52.97,4,[PASS],<NA>,<NA>,<NA>,,splice_donor_variant,HIGH,[1.0],52.97,Review Required
3,NA21144,chr9,109532943,GTTT,[G],52.73,11,[PASS],<NA>,<NA>,<NA>,YBX1P6,splice_acceptor_variant,HIGH,[0.909],52.73,Review Required
4,NA21144,chr3,178335448,CTATATATA,[C],51.68,11,[PASS],<NA>,<NA>,<NA>,,splice_acceptor_variant,HIGH,[0.727],51.68,Review Required
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,NA21135,chr7,72744550,C,[CA],146.82,35,[PASS],<NA>,<NA>,<NA>,TYW1B,frameshift_variant,HIGH,[1.0],146.82,Review Required
517,NA21144,chr7,72744550,C,[CA],146.82,35,[PASS],<NA>,<NA>,<NA>,TYW1B,frameshift_variant,HIGH,[1.0],146.82,Review Required
518,NA21144,chr9,104605110,TGTTA,[T],146.74,34,[PASS],<NA>,<NA>,<NA>,OR13C2,frameshift_variant,HIGH,[1.0],146.74,Review Required
519,NA21135,chr19,52383891,TA,[T],146.31,33,[PASS],<NA>,<NA>,<NA>,ZNF880,frameshift_variant,HIGH,[1.0],146.31,Review Required


In [22]:
vep_filter_comp.to_csv("comprehensive_clinrelevant_results.csv", index=False)
print("Exported to comprehensive_clinrelevant_results.csv")

Exported to comprehensive_clinrelevant_results.csv


In [20]:
## Comprehensive Query - Only PASS Variants
comprehensive_pass_only = f"""
SELECT 
    v.sampleid,
    v.contigname,
    v.start,
    v.referenceallele,
    v.alternatealleles,
    v.qual,
    v.depth,
    v.filters,
    
    -- ClinVar Information
    a.names as clinvar_variation_id,
    a.attributes['CLNSIG'] as clinical_significance,
    a.attributes['CLNDN'] as disease_name,
    a.attributes['GENEINFO'] as clinvar_gene_info,
    a.attributes['CLNREVSTAT'] as review_status,
    a.attributes['RS'] as dbsnp_id,
    a.attributes['ALLELEID'] as clinvar_allele_id,
    
    -- VEP Annotations
    CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].symbol END as vep_gene_symbol,
    CASE WHEN cardinality(v.annotations.vep) > 0 AND cardinality(v.annotations.vep[1].consequence) > 0 
         THEN v.annotations.vep[1].consequence[1] END as vep_consequence,
    CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].impact END as vep_impact,
    CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].biotype END as vep_biotype,
    CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].hgvsc END as hgvs_coding,
    CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].hgvsp END as hgvs_protein,
    
    -- Quality Metrics
    v.information['af'] as allele_frequency,
    v.information['dp'] as total_depth,
    v.information['ac'] as allele_count,
    v.information['an'] as allele_number,
    v.information['fs'] as fisher_strand_bias,
    v.information['sor'] as strand_odds_ratio,
    v.information['mq'] as mapping_quality,
    
    -- Clinical Assessment
    CASE 
        WHEN a.attributes['CLNSIG'] IN ('Pathogenic', 'Likely_pathogenic') 
             AND (CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].impact END) IN ('HIGH', 'MODERATE')
        THEN 'High Clinical Concern'
        WHEN a.attributes['CLNSIG'] = 'Uncertain_significance' 
             AND (CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].impact END) = 'HIGH'
        THEN 'Moderate Clinical Concern'
        WHEN a.attributes['CLNSIG'] IN ('Benign', 'Likely_benign')
        THEN 'Low Clinical Concern'
        ELSE 'Unknown Clinical Significance'
    END as clinical_assessment

FROM "default"."{var_store['name']}" v
LEFT JOIN "default"."{ann_store['name']}" a ON (
    REPLACE(v.contigname, 'chr', '') = a.contigname
    AND v.start = a.start
    AND v.referenceallele = a.referenceallele
    AND v.alternatealleles = a.alternatealleles
)
WHERE v.qual > 30  
    AND contains(v.filters, 'PASS')  -- CORRECT: Only variants that passed filters
    AND (
        a.attributes['CLNSIG'] IS NOT NULL  -- Has ClinVar annotation
        OR (cardinality(v.annotations.vep) > 0 AND v.annotations.vep[1].impact IN ('HIGH', 'MODERATE'))  -- High/Moderate impact
    )
ORDER BY 
    CASE 
        WHEN a.attributes['CLNSIG'] = 'Pathogenic' THEN 1
        WHEN a.attributes['CLNSIG'] = 'Likely_pathogenic' THEN 2
        WHEN a.attributes['CLNSIG'] = 'Uncertain_significance' THEN 3
        ELSE 4
    END,
    v.qual ASC
"""

In [21]:
vep_filter_pass = wr.athena.read_sql_query(
    comprehensive_pass_only, 
    database='<YOUR_AWS_PROFILE>', 
    workgroup=athena_workgroup['Name'])
vep_filter_pass

,sampleid,contigname,start,referenceallele,alternatealleles,qual,depth,filters,clinvar_variation_id,clinical_significance,...,hgvs_coding,hgvs_protein,allele_frequency,total_depth,allele_count,allele_number,fisher_strand_bias,strand_odds_ratio,mapping_quality,clinical_assessment
0,NA21135,chr19,17559373,G,[A],42.06,40,[PASS],[3728920],Benign,...,,,[0.35],<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Low Clinical Concern
1,NA21135,chr3,10337938,A,[G],42.06,38,[PASS],[1262156],Benign,...,,,[0.342],<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Low Clinical Concern
2,NA21144,chr11,17442986,T,[C],42.06,40,[PASS],[1283124],Benign,...,,,[0.35],<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Low Clinical Concern
3,NA21135,chr13,47949906,A,[T],42.06,40,[PASS],[671130],Benign,...,,,[0.35],<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Low Clinical Concern
4,NA21135,chr11,72105045,T,[A],42.06,40,[PASS],[682807],Benign,...,,,[0.35],<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Low Clinical Concern
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101105,NA21144,chr3,64541265,A,[C],161.26,38,[PASS],[1222970],Benign,...,,,[1.0],<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Low Clinical Concern
101106,NA21135,chr1,154171583,T,[TAG],161.26,38,[PASS],[670244],Benign,...,,,[1.0],<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Low Clinical Concern
101107,NA21135,chr16,88643419,A,[G],161.26,38,[PASS],[198042],Benign,...,,,[1.0],<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Low Clinical Concern
101108,NA21144,chr2,233816855,T,[C],161.26,38,[PASS],None,<NA>,...,,,[1.0],<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Unknown Clinical Significance


In [38]:
## Gene-centric query
schema_analysis_query = f"""SHOW COLUMNS FROM  "default"."{var_store['name']}" """

In [ ]:
!aws glue get-table \
    --database-name "default" \
    --name "genomicsvariantstore" \
    --query 'Table.StorageDescriptor.Columns[*].{Name:Name,Type:Type,Comment:Comment}' \
    --output json

In [ ]:
!aws glue get-table \
    --database-name "default" \
    --name "genomicsannotationstore" \
    --query 'Table.StorageDescriptor.Columns[*].{Name:Name,Type:Type,Comment:Comment}' \
    --output json

In [45]:
gene_analysis_query = f"""
WITH gene_variants AS (
    SELECT 
        v.sampleid,
        v.contigname,
        v.start,
        v.referenceallele,
        v.alternatealleles,
        v.qual,
        v.depth,
        v.annotations,
        a.attributes,
        -- Extract gene symbol from GENEINFO or VEP annotations
        COALESCE(
            CASE 
                WHEN a.attributes['GENEINFO'] IS NOT NULL 
                THEN split_part(a.attributes['GENEINFO'], ':', 1)
                ELSE NULL
            END,
            CASE 
                WHEN cardinality(v.annotations.vep) > 0 
                THEN v.annotations.vep[1].symbol
                ELSE NULL
            END
        ) as gene_symbol,
        
        -- Extract clinical significance
        a.attributes['CLNSIG'] as clinical_significance,
        
        -- Extract VEP impact and consequence
        CASE 
            WHEN cardinality(v.annotations.vep) > 0 
            THEN v.annotations.vep[1].impact
            ELSE NULL
        END as vep_impact,
        
        CASE 
            WHEN cardinality(v.annotations.vep) > 0 
            AND cardinality(v.annotations.vep[1].consequence) > 0
            THEN v.annotations.vep[1].consequence[1]
            ELSE NULL
        END as vep_consequence
        
    FROM "default"."{var_store['name']}" v
    LEFT JOIN "default"."{ann_store['name']}" a ON (
        REPLACE(v.contigname, 'chr', '') = REPLACE(a.contigname, 'chr', '')
        AND v.start = a.start
        AND v.referenceallele = a.referenceallele
        AND v.alternatealleles[1] = a.alternatealleles[1]
    )
)

SELECT 
    gene_symbol,
    
    COUNT(*) as total_variants,
    
    COUNT(CASE 
        WHEN clinical_significance IN ('Pathogenic', 'Likely_pathogenic', 'Pathogenic/Likely_pathogenic') 
        THEN 1 
    END) as pathogenic_variants,
    
    COUNT(CASE 
        WHEN clinical_significance = 'Uncertain_significance' 
        THEN 1 
    END) as vus_variants,
    
    COUNT(CASE 
        WHEN vep_impact = 'HIGH' 
        THEN 1 
    END) as high_impact_variants,
    
    COUNT(CASE 
        WHEN vep_impact = 'MODERATE' 
        THEN 1 
    END) as moderate_impact_variants,
    
    -- Sample distribution
    COUNT(DISTINCT sampleid) as samples_affected,
    
    -- Quality metrics
    ROUND(AVG(CAST(qual as DOUBLE)), 2) as avg_quality,
    ROUND(AVG(CAST(depth as DOUBLE)), 2) as avg_depth,
    
    -- Consequence types (using array_agg with DISTINCT)
    array_join(
        array_distinct(
            array_agg(
                CASE 
                    WHEN vep_consequence IS NOT NULL 
                    THEN vep_consequence 
                END
            )
        ), 
        ', '
    ) as consequence_types,
    
    -- Additional useful metrics
    COUNT(CASE 
        WHEN clinical_significance IN ('Benign', 'Likely_benign') 
        THEN 1 
    END) as benign_variants,
    
    ROUND(
        COUNT(CASE 
            WHEN clinical_significance IN ('Pathogenic', 'Likely_pathogenic', 'Pathogenic/Likely_pathogenic') 
            THEN 1 
        END) * 100.0 / COUNT(*), 
        2
    ) as pathogenic_percentage

FROM gene_variants
WHERE gene_symbol IS NOT NULL 
    AND gene_symbol != ''
GROUP BY gene_symbol
HAVING COUNT(*) >= 2  -- Genes with multiple variants
ORDER BY pathogenic_variants DESC, high_impact_variants DESC, total_variants DESC
LIMIT 100
"""

In [46]:
vep_gene_anal = wr.athena.read_sql_query(
    gene_analysis_query, 
    database='<YOUR_AWS_PROFILE>', 
    workgroup=athena_workgroup['Name'])
vep_gene_anal

,gene_symbol,total_variants,pathogenic_variants,vus_variants,high_impact_variants,moderate_impact_variants,samples_affected,avg_quality,avg_depth,consequence_types,benign_variants,pathogenic_percentage
0,ZNF717,1750,2,0,0,4,2,72.43,53.34,"intron_variant, downstream_gene_variant, non_c...",2,0.11
1,GBP1,62,2,0,0,3,2,54.57,33.89,"downstream_gene_variant, splice_polypyrimidine...",0,3.23
2,PRB3,40,1,0,2,6,2,81.19,26.15,"frameshift_variant, missense_variant, intron_v...",1,2.50
3,SLC9B1,560,1,0,1,2,2,48.05,32.73,"3_prime_UTR_variant, intron_variant, upstream_...",0,0.18
4,CLASP1,1027,1,0,0,0,2,53.10,30.26,"downstream_gene_variant, intron_variant, 3_pri...",3,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...
95,TPM1-AS,448,0,0,2,0,2,74.25,32.95,"downstream_gene_variant, intron_variant, splic...",0,0.00
96,VWDE,446,0,0,2,21,2,75.95,30.87,"downstream_gene_variant, 3_prime_UTR_variant, ...",0,0.00
97,ZNF443,416,0,0,2,17,2,55.78,30.53,"downstream_gene_variant, 3_prime_UTR_variant, ...",0,0.00
98,COL6A5,406,0,0,2,11,2,105.17,31.33,"upstream_gene_variant, intron_variant, missens...",2,0.00


In [49]:
gene_analysis_query = f"""
SELECT 
    COALESCE(
        split_part(a.attributes['GENEINFO'], ':', 1),
        CASE 
            WHEN cardinality(v.annotations.vep) > 0 
            THEN v.annotations.vep[1].symbol
        END
    ) as gene_symbol,
    
    COUNT(*) as total_variants,
    COUNT(CASE WHEN a.attributes['CLNSIG'] IN ('Pathogenic', 'Likely_pathogenic') THEN 1 END) as pathogenic_variants,
    COUNT(CASE WHEN a.attributes['CLNSIG'] = 'Uncertain_significance' THEN 1 END) as vus_variants,
    COUNT(CASE WHEN v.annotations.vep[1].impact = 'HIGH' THEN 1 END) as high_impact_variants,
    
    COUNT(DISTINCT v.sampleid) as samples_affected,
    ROUND(AVG(CAST(v.qual as DOUBLE)), 2) as avg_quality,
    ROUND(AVG(CAST(v.depth as DOUBLE)), 2) as avg_depth

FROM "default"."{var_store['name']}" v
LEFT JOIN "default"."{ann_store['name']}" a ON (
    REPLACE(v.contigname, 'chr', '') = REPLACE(a.contigname, 'chr', '')
    AND v.start = a.start
    AND v.referenceallele = a.referenceallele
    AND v.alternatealleles[1] = a.alternatealleles[1]
)
WHERE (a.attributes['GENEINFO'] IS NOT NULL OR cardinality(v.annotations.vep) > 0)
GROUP BY 1
HAVING COUNT(*) >= 2  -- Use the actual aggregate function instead of alias
ORDER BY pathogenic_variants DESC, high_impact_variants DESC
LIMIT 100
"""

In [50]:
vep_gene_anal = wr.athena.read_sql_query(
    gene_analysis_query, 
    database='<YOUR_AWS_PROFILE>', 
    workgroup=athena_workgroup['Name'])
vep_gene_anal

,gene_symbol,total_variants,pathogenic_variants,vus_variants,high_impact_variants,samples_affected,avg_quality,avg_depth
0,ZNF717,1750,2,0,0,2,72.43,53.34
1,GBP1,62,2,0,0,2,54.57,33.89
2,PRB3,40,1,0,2,2,81.19,26.15
3,SLC9B1,560,1,0,1,2,48.05,32.73
4,CYP21A2,19,1,0,0,2,42.56,12.47
...,...,...,...,...,...,...,...,...
95,ZNF22-AS1,622,0,0,2,2,66.72,31.30
96,NUDT11,16,0,0,2,2,103.53,19.44
97,TMEM216,27,0,0,2,2,124.30,33.81
98,USP29,95,0,0,2,2,83.65,30.45


In [53]:
clinical_priority_query = f"""
WITH variant_annotations AS (
    SELECT 
        v.sampleid,
        v.contigname,
        v.start,
        v.referenceallele,
        v.alternatealleles[1] as alternate_allele,
        v.qual,
        v.depth,
        v.information['af'] as allele_frequency,
        v.filters[1] as filter_status,
        
        -- Extract VEP annotations safely
        CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].symbol END as vep_gene,
        CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].impact END as vep_impact,
        CASE WHEN cardinality(v.annotations.vep) > 0 AND cardinality(v.annotations.vep[1].consequence) > 0 
             THEN v.annotations.vep[1].consequence[1] END as vep_consequence,
        
        -- Clinical annotations
        a.attributes['CLNSIG'] as clinvar_significance,
        a.attributes['CLNDN'] as associated_disease,
        a.attributes['CLNREVSTAT'] as review_status,
        split_part(a.attributes['GENEINFO'], ':', 1) as clinvar_gene
        
    FROM "default"."{var_store['name']}" v
    LEFT JOIN "default"."{ann_store['name']}" a ON (
        REPLACE(v.contigname, 'chr', '') = REPLACE(a.contigname, 'chr', '')
        AND v.start = a.start
        AND v.referenceallele = a.referenceallele
        AND v.alternatealleles[1] = a.alternatealleles[1]
    )
)

SELECT 
    sampleid,
    CONCAT(contigname, ':', CAST(start as VARCHAR), ':', referenceallele, '>', alternate_allele) as variant_id,
    COALESCE(clinvar_gene, vep_gene) as gene_symbol,
    vep_consequence as consequence,
    vep_impact as impact,
    clinvar_significance,
    associated_disease,
    review_status,
    qual,
    depth,
    allele_frequency,
    
    -- Priority scoring
    CASE 
        WHEN clinvar_significance = 'Pathogenic' AND vep_impact = 'HIGH' THEN 10
        WHEN clinvar_significance = 'Pathogenic' AND vep_impact = 'MODERATE' THEN 9
        WHEN clinvar_significance = 'Likely_pathogenic' AND vep_impact = 'HIGH' THEN 8
        WHEN clinvar_significance = 'Likely_pathogenic' AND vep_impact = 'MODERATE' THEN 7
        WHEN clinvar_significance = 'Uncertain_significance' AND vep_impact = 'HIGH' THEN 6
        WHEN vep_impact = 'HIGH' THEN 5
        WHEN clinvar_significance = 'Uncertain_significance' AND vep_impact = 'MODERATE' THEN 4
        ELSE 1
    END as priority_score

FROM variant_annotations
WHERE qual > 30 
    AND filter_status = 'PASS'
    AND (clinvar_significance IS NOT NULL OR vep_impact IN ('HIGH', 'MODERATE'))
ORDER BY priority_score DESC, qual DESC
LIMIT 1000
"""

In [54]:
vep_clin_anal = wr.athena.read_sql_query(
    clinical_priority_query, 
    database='<YOUR_AWS_PROFILE>', 
    workgroup=athena_workgroup['Name'])
vep_clin_anal

,sampleid,variant_id,gene_symbol,consequence,impact,clinvar_significance,associated_disease,review_status,qual,depth,allele_frequency,priority_score
0,NA21144,chr12:11267399:A>AG,PRB3,frameshift_variant,HIGH,Pathogenic,PRB3M(NULL),no_assertion_criteria_provided,48.85,17,[0.412],10
1,NA21135,chr6:31356750:G>GGA,HLA-B,frameshift_variant,HIGH,<NA>,<NA>,<NA>,860.80,42,"[0.476, 0.524]",5
2,NA21135,chr6:31356710:C>CGGAG,HLA-B,frameshift_variant,HIGH,<NA>,<NA>,<NA>,665.90,33,"[0.455, 0.545]",5
3,NA21135,chr6:32666522:GTA>G,HLA-DQB1,frameshift_variant,HIGH,<NA>,<NA>,<NA>,616.17,44,"[0.568, 0.432]",5
4,NA21135,chr6:31412383:G>GCT,MICA,frameshift_variant,HIGH,Benign,not_specified,"criteria_provided,_single_submitter",570.50,36,"[0.611, 0.389]",5
...,...,...,...,...,...,...,...,...,...,...,...,...
995,NA21135,chr13:25096662:T>TG,PABPC3,frameshift_variant,HIGH,<NA>,<NA>,<NA>,35.74,38,[0.316],5
996,NA21144,chr22:22747453:ACT>A,IGLV3-16,frameshift_variant,HIGH,<NA>,<NA>,<NA>,35.33,19,[0.211],5
997,NA21135,chr15:22466822:T>C,GOLGA6L22,stop_lost,HIGH,<NA>,<NA>,<NA>,35.32,45,[0.711],5
998,NA21144,chr22:22196263:A>ACCCCAGTCTCAGGGGAGTGTTCGGCGGA...,IGLV6-57,frameshift_variant,HIGH,<NA>,<NA>,<NA>,34.41,22,[0.227],5


In [55]:
population_analysis_query = f"""
WITH variant_data AS (
    SELECT 
        v.sampleid,
        v.contigname,
        v.start,
        v.referenceallele,
        v.alternatealleles[1] as alternate_allele,
        v.qual,
        v.depth,
        v.information,
        
        -- Extract VEP annotations safely
        CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].symbol END as vep_gene,
        CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].impact END as vep_impact,
        CASE WHEN cardinality(v.annotations.vep) > 0 AND cardinality(v.annotations.vep[1].consequence) > 0 
             THEN v.annotations.vep[1].consequence[1] END as vep_consequence,
        
        -- Clinical annotations
        a.attributes['CLNSIG'] as clinical_significance,
        split_part(a.attributes['GENEINFO'], ':', 1) as clinvar_gene,
        
        -- Parse numeric values safely
        TRY_CAST(v.information['af'] as DOUBLE) as allele_frequency,
        TRY_CAST(v.information['dp'] as INTEGER) as total_depth,
        TRY_CAST(v.information['mq'] as DOUBLE) as mapping_quality
        
    FROM "default"."{var_store['name']}" v
    LEFT JOIN "default"."{ann_store['name']}" a ON (
        REPLACE(v.contigname, 'chr', '') = REPLACE(a.contigname, 'chr', '')
        AND v.start = a.start
        AND v.referenceallele = a.referenceallele
        AND v.alternatealleles[1] = a.alternatealleles[1]
    )
    WHERE v.information['af'] IS NOT NULL
)

SELECT 
    sampleid,
    COALESCE(clinvar_gene, vep_gene) as gene_symbol,
    contigname,
    start,
    referenceallele,
    alternate_allele,
    qual,
    depth,
    allele_frequency,
    total_depth,
    mapping_quality,
    clinical_significance,
    vep_impact,
    vep_consequence as consequence,
    
    -- Quality flags
    CASE 
        WHEN qual > 100 AND depth > 20 THEN 'High Quality'
        WHEN qual > 50 AND depth > 10 THEN 'Medium Quality'
        ELSE 'Low Quality'
    END as quality_tier,
    
    -- Rarity assessment
    CASE 
        WHEN allele_frequency < 0.001 THEN 'Very Rare'
        WHEN allele_frequency < 0.01 THEN 'Rare'
        WHEN allele_frequency < 0.05 THEN 'Uncommon'
        WHEN allele_frequency IS NOT NULL THEN 'Common'
        ELSE 'Unknown'
    END as rarity_category,
    
    -- Additional population genetics metrics
    CASE 
        WHEN allele_frequency IS NOT NULL AND allele_frequency > 0 
        THEN ROUND(-LOG10(allele_frequency), 2)
        ELSE NULL
    END as rarity_score,
    
    -- Hardy-Weinberg expected heterozygote frequency (2pq)
    CASE 
        WHEN allele_frequency IS NOT NULL AND allele_frequency > 0 AND allele_frequency < 1
        THEN ROUND(2 * allele_frequency * (1 - allele_frequency), 4)
        ELSE NULL
    END as expected_het_frequency

FROM variant_data
WHERE allele_frequency IS NOT NULL
ORDER BY allele_frequency ASC, qual DESC
LIMIT 10000
"""

In [56]:
vep_pop_anal = wr.athena.read_sql_query(
    population_analysis_query, 
    database='<YOUR_AWS_PROFILE>', 
    workgroup=athena_workgroup['Name'])
vep_pop_anal

,sampleid,gene_symbol,contigname,start,referenceallele,alternate_allele,qual,depth,allele_frequency,total_depth,mapping_quality,clinical_significance,vep_impact,consequence,quality_tier,rarity_category,rarity_score,expected_het_frequency


In [63]:
sample_comparison_query = f"""
WITH sample_variants AS (
    SELECT 
        v.sampleid,
        v.qual,
        v.depth,
        v.referenceallele,
        v.alternatealleles[1] as alternate_allele,
        v.filters[1] as filter_status,
        
        -- Extract VEP annotations safely
        CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].symbol END as vep_gene,
        CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].impact END as vep_impact,
        
        -- Clinical annotations
        a.attributes['CLNSIG'] as clinical_significance,
        split_part(a.attributes['GENEINFO'], ':', 1) as clinvar_gene
        
    FROM "default"."{var_store['name']}" v
    LEFT JOIN "default"."{ann_store['name']}" a ON (
        REPLACE(v.contigname, 'chr', '') = REPLACE(a.contigname, 'chr', '')
        AND v.start = a.start
        AND v.referenceallele = a.referenceallele
        AND v.alternatealleles[1] = a.alternatealleles[1]
    )
    WHERE v.filters[1] = 'PASS'
)

SELECT 
    sampleid,
    COUNT(*) as total_variants,
    
    -- Clinical significance counts
    COUNT(CASE WHEN clinical_significance = 'Pathogenic' THEN 1 END) as pathogenic_count,
    COUNT(CASE WHEN clinical_significance = 'Likely_pathogenic' THEN 1 END) as likely_pathogenic_count,
    COUNT(CASE WHEN clinical_significance = 'Uncertain_significance' THEN 1 END) as vus_count,
    COUNT(CASE WHEN clinical_significance IN ('Benign', 'Likely_benign') THEN 1 END) as benign_count,
    
    -- Impact counts
    COUNT(CASE WHEN vep_impact = 'HIGH' THEN 1 END) as high_impact_count,
    COUNT(CASE WHEN vep_impact = 'MODERATE' THEN 1 END) as moderate_impact_count,
    COUNT(CASE WHEN vep_impact = 'LOW' THEN 1 END) as low_impact_count,
    COUNT(CASE WHEN vep_impact = 'MODIFIER' THEN 1 END) as modifier_impact_count,
    
    -- Quality metrics
    ROUND(AVG(CAST(qual as DOUBLE)), 2) as avg_quality,
    ROUND(AVG(CAST(depth as DOUBLE)), 2) as avg_depth,
    MIN(qual) as min_quality,
    MAX(qual) as max_quality,
    
    -- Gene diversity
    COUNT(DISTINCT COALESCE(clinvar_gene, vep_gene)) as unique_genes_affected,
    
    -- Variant type distribution
    COUNT(CASE 
        WHEN LENGTH(referenceallele) = 1 AND LENGTH(alternate_allele) = 1 
        THEN 1 
    END) as snv_count,
    
    COUNT(CASE 
        WHEN LENGTH(referenceallele) > LENGTH(alternate_allele) 
        THEN 1 
    END) as deletion_count,
    
    COUNT(CASE 
        WHEN LENGTH(referenceallele) < LENGTH(alternate_allele) 
        THEN 1 
    END) as insertion_count,
    
    -- Clinical significance percentage
    ROUND(
        COUNT(CASE WHEN clinical_significance IN ('Pathogenic', 'Likely_pathogenic') THEN 1 END) * 100.0 / COUNT(*), 
        2
    ) as pathogenic_percentage

FROM sample_variants
GROUP BY sampleid
ORDER BY pathogenic_count DESC, likely_pathogenic_count DESC, total_variants DESC
LIMIT 100
"""

In [64]:
vep_sample_anal = wr.athena.read_sql_query(
    sample_comparison_query, 
    database='<YOUR_AWS_PROFILE>', 
    workgroup=athena_workgroup['Name'])
vep_sample_anal

,sampleid,total_variants,pathogenic_count,likely_pathogenic_count,vus_count,benign_count,high_impact_count,moderate_impact_count,low_impact_count,modifier_impact_count,avg_quality,avg_depth,min_quality,max_quality,unique_genes_affected,snv_count,deletion_count,insertion_count,pathogenic_percentage
0,NA21135,4978141,4,1,267,42258,520,9841,16752,4951028,86.94,33.60,10.41,3137.61,31214,4004174,500219,473738,0.0
1,NA21144,5013114,2,1,282,42323,560,10001,16656,4985897,80.34,28.46,10.41,3154.34,31085,4036053,503103,473951,0.0


In [65]:
population_analysis_query = f"""
SELECT 
    v.sampleid,
    COALESCE(
        CASE 
            WHEN a.attributes['GENEINFO'] IS NOT NULL 
            THEN split_part(a.attributes['GENEINFO'], ':', 1)
            ELSE NULL
        END,
        CASE 
            WHEN cardinality(v.annotations.vep) > 0 
            THEN v.annotations.vep[1].symbol
            ELSE NULL
        END
    ) as gene_symbol,
    
    v.contigname,
    v.start,
    v.referenceallele,
    v.alternatealleles[1] as alternate_allele,
    v.qual,
    v.depth,
    
    -- Try both attributes and information for AF
    COALESCE(
        TRY_CAST(v.attributes['AF'] as DOUBLE),
        TRY_CAST(v.information['af'] as DOUBLE),
        TRY_CAST(v.information['AF'] as DOUBLE)
    ) as allele_frequency,
    
    TRY_CAST(v.attributes['DP'] as INTEGER) as total_depth,
    TRY_CAST(v.attributes['MQ'] as DOUBLE) as mapping_quality,
    
    a.attributes['CLNSIG'] as clinical_significance

FROM "default"."{var_store['name']}" v
LEFT JOIN "default"."{ann_store['name']}" a ON (
    REPLACE(v.contigname, 'chr', '') = REPLACE(a.contigname, 'chr', '')
    AND v.start = a.start
    AND v.referenceallele = a.referenceallele
    AND v.alternatealleles[1] = a.alternatealleles[1]
)
WHERE (v.attributes['AF'] IS NOT NULL 
       OR v.information['af'] IS NOT NULL 
       OR v.information['AF'] IS NOT NULL)
LIMIT 100
"""

In [66]:
vep_pop_anal = wr.athena.read_sql_query(
    population_analysis_query, 
    database='<YOUR_AWS_PROFILE>', 
    workgroup=athena_workgroup['Name'])
vep_pop_anal

,sampleid,gene_symbol,contigname,start,referenceallele,alternate_allele,qual,depth,allele_frequency,total_depth,mapping_quality,clinical_significance
0,NA21135,HISLA,chr14,88080795,T,C,50.00,38,0.5,39,250.00,<NA>
1,NA21135,,chr8,53341508,T,C,177.35,45,1.0,45,63.52,<NA>
2,NA21135,HISLA,chr14,88083818,A,G,42.20,30,0.5,31,250.00,<NA>
3,NA21135,HISLA,chr14,88088056,T,C,49.74,34,0.5,34,250.00,<NA>
4,NA21135,,chr8,53346928,A,C,119.21,24,1.0,24,224.51,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...
95,NA21135,LINC02984,chr8,53462821,A,G,48.98,34,0.5,34,188.63,<NA>
96,NA21135,KCNK10,chr14,88221066,C,T,50.00,27,0.5,27,230.18,<NA>
97,NA21135,LINC02984,chr8,53467468,T,C,50.00,26,0.5,26,194.74,<NA>
98,NA21135,LINC02984,chr8,53470372,G,A,50.00,41,0.5,41,250.00,<NA>
